In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/MyDrive/proj

In [3]:
import sys
sys.path.append('/content/gdrive/MyDrive/proj')

## autoencoder 

In [4]:
from generate_model import Designed_model as GM
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Reshape,Dense, GaussianNoise,Lambda,Dropout
from keras.models import Model
from keras import regularizers
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

class aphl(GM):
   def __init__(self,data_path):
      super(aphl, self).__init__(data_path)
   
   def build_model_and_get_data(self):
      if self.X_train == None : self.set_data() 
      self.model = self.CNN_model_article(
         self.X_train[0].shape, 
         self.y_train[0].shape[0])
      self.compile_model(self.model)
      return 
   
   def train_gpu(self,epochs =20,batch_size=1024 ,compile = True): 
       with tf.device(tf.test.gpu_device_name()):
          self.train(epochs=epochs, batch_size=batch_size)    

   def train(self, epochs =20,batch_size=1024 ,compile = True, get_data_build =True):
      filepath = self.data_path
      if get_data_build : self.build_model_and_get_data()
      if compile : self.model = self.compile_model(self.model)
      history = self.model.fit(self.X_train,self.y_train,
              validation_data=(self.X_test, self.y_test),
              callbacks = [
                    keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
                    keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')],
              epochs=epochs, batch_size = batch_size, verbose=2 )
      
      return history
   
   
   def set_data(self , limit = None) -> None:
      ##read from directory
      df = pd.read_pickle( self.data_path + "RML2016.10a_dict.pkl")
      d = pd.DataFrame(df.keys())
      snrs,mods = d[0].unique() , d[1].unique().tolist()
      if type (snrs[1]) == str : 
         snrs,mods = d[1].unique() , d[0].unique().tolist()
      ## creat label 
      X = []  
      label = []
      for key , value in df.items():
         label.extend([key]*value.shape[0])
         X.append(value)
      X = np.vstack(X)
      
      ## make one-hot label
      encoded = to_categorical([i for i in range(len(mods))]) 
      
      # seperate data for train and test
      y = label 
      
      if self.limit_check(limit,X):
            X = X[:limit]
            y = X[:limit] 

      X_mean = self.mean_data(X)
      self._mean = X_mean
      
      X_scale = self.scale(X)
      self._mean = X_scale
      
      X = self.transform(X,X_mean,X_scale)

      X_train, X_test, y_train, y_test = train_test_split(
                                             X, y, test_size=0.4, random_state=42)

      SNR_train = np.array([lbl[1] for lbl in y_train])
      y_train = np.array([self.mods_to_one_hot(mods,lbl[0],encoded) for lbl in y_train])
      y_test  = np.array([self.mods_to_one_hot(mods,lbl[0],encoded) for lbl in y_test])

      sorted_arg_SNR_descended = SNR_train.argsort()[::-1]
      X_train = X_train[sorted_arg_SNR_descended]
      y_train = y_train[sorted_arg_SNR_descended]

      X = X.astype('float64')
      y_train = y_train.astype('float64')
      y_test = y_test.astype('float64')

      self.X_train , self.y_train, self.X_test, self.y_test = X_train ,y_train, X_test,y_test
   
   @staticmethod
   def compile_model(model):
      model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=0.001),
                    loss= tf.keras.losses.CategoricalCrossentropy(),
                    metrics=["accuracy"],
                )  
      return model
   
   @staticmethod
   def mods_to_one_hot (mods,inp,encoded):
     
    return encoded[mods.index(inp)]


   @staticmethod
   def complex_to_array(dataset):
      shape = dataset.shape
      temp = dataset.reshape([*shape,1])
      real_num , imag_num = np.real(temp) , np.imag(temp)
      array_form_of_complex = np.concatenate((real_num,imag_num),axis=len(shape))
      return array_form_of_complex 

   @staticmethod
   def transform(x,mean,scale):
     return (x-mean)/scale


   @staticmethod 
   def mean_data(data):
    for i in range(len(data.shape)-2):
        if i == 0 : mean = np.mean(data,axis=0)
        mean = np.mean(mean,axis=0)   
    return mean
    
   @staticmethod 
   def scale(data):
    max = np.max(np.abs(data))
    return  max 

   
   @staticmethod
   def CNN_model_article(input_shape = (2,128) , output_shape=11):

    input_signal = Input(shape = input_shape)
    reshape = Reshape((2,128,1))(input_signal)
    conv_1 = Conv2D(128 , kernel_size = (2, 8), activation='relu' )(reshape)
    max_pool_1 = MaxPooling2D (pool_size=(1,2), strides = 2,data_format ='channels_last')(conv_1)
    
    conv_2 = Conv2D(64, kernel_size =(1, 16), activation='relu')(max_pool_1)
    max_pool_2 = MaxPooling2D (pool_size=(1,2), strides = 2)(conv_2)

    flatten = Flatten()(max_pool_2)

    dense_1 = Dense(128, activation='relu')(flatten)
    dense_2 = Dense(64, activation='relu')(dense_1)
    dense_3 = Dense(32, activation='relu')(dense_2)
    dense_4 = Dense(output_shape, activation='softmax')(dense_3)

    return Model(inputs = input_signal, outputs = dense_4)  

## DeepMIMO

In [5]:
!pip install DeepMIMO

In [6]:
import DeepMIMO
import numpy as np 

In [7]:
import DeepMIMO
import numpy as np 
from generate_model import Designed_model as GM
from sklearn.model_selection import train_test_split
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers , Input , Sequential 
from tensorflow.keras.layers import Flatten , Dense , Reshape 
import keras.backend as K


class Deep_MIMO(GM):
   def __init__(self,data_path ,active_BS = 64,
                                user_row_first = 1,
                                user_row_last = 10,
                                bandwidth = 0.05,
                                subcarriers = 64 ,
                                subcarriers_sampling = 1 ,
                                subcarriers_limit = 32,
                                num_paths = 1,
                                num_set_antenna_M = 8):
      super(Deep_MIMO, self).__init__(data_path)
      for i,(key,value) in enumerate(locals().items()):
          if i < 2 : continue
          setattr(self,key,value)


   
   def build_model_and_get_data(self,MLP_DroupOut = True):
      if type (self.X_train) == type(None): self.set_data() 

      input_shape = self.X_train[0].shape
      output_shape = self.y_train[0].shape

      if MLP_DroupOut : self.model = self.MlP_with_dropouts (input_shape , output_shape)
      else : self.model = self.MLP( input_shape , output_shape)
      self.compile_model (self.model, self.MSE_loss )
      
      return self.model
   

   def train_gpu(self,epochs =17,batch_size=100 ,compile = True) -> None: 
       with tf.device(tf.test.gpu_device_name()):
          self.train(epochs=epochs, batch_size=batch_size)    

   def train(self, epochs =17,batch_size=100 ,compile = True, get_data_build =True) -> None:
      if get_data_build : self.build_model_and_get_data()
      if compile : self.model = self.compile_model (self.model, self.MSE_loss )
      history = self.model.fit(self.X_train,self.y_train,
              validation_data=(self.X_test, self.y_test),
              epochs=epochs, batch_size = batch_size, verbose=2 )
      return history
   
   
   def set_data(self , limit = None) -> None:

      dataset_up = self.get_MIMO_data('I1_2p4')
      dataset_down = self.get_MIMO_data('I1_2p4')

      dataset_up = self.draw_necessary_inf_in_desired_shape (dataset_up)
      dataset_down = self.draw_necessary_inf_in_desired_shape (dataset_down)

      data_up = self.complex_to_array(dataset_up)
      data_down = self.complex_to_array(dataset_down)

      
      X_mean = self.mean_data(data_up)
      y_mean = self.mean_data(data_down)
      self._mean = X_mean
      
      X_scale = self.scale(data_up)
      y_scale = self.scale (data_down)
      self._scale = X_scale
      
      norm_data_up = self.transform(data_up,X_mean,X_scale)
      norm_data_down = self.transform(data_down,y_mean,y_scale)

      X = np.array([u*self.make_mask(u,num_set_antenna_M = self.num_set_antenna_M) for u in norm_data_up[:]])
      y = norm_data_down

      X = X.astype('float64')
      y = y.astype('float64')
      X_train, X_test, y_train, y_test = train_test_split(
                                              X, y, test_size=0.33, random_state=42)
      
      self.X_train , self.y_train, self.X_test, self.y_test = X_train ,y_train, X_test,y_test


   
   def compile_model(self, model, func ):
      model.compile(
                      optimizer=keras.optimizers.Adam(learning_rate=0.001),
                      loss= func,
                      metrics=["accuracy"],
                  )  
      return model  

   def model_eval (batch_size = 100):
      self.model.evaluate(self.X_test, self.y_test, batch_size=batch_size, verbose=2)

   
   @staticmethod
   def draw_necessary_inf_in_desired_shape(data):
     #just using one RX antenna data and one TX antenna form all antennas information  
      data = [data[i]['user']["channel"][:][:,0,0] for i in range(len(data))]
      data = np.array(data)
      print(data.shape)
      #changing the data into form of : [num_user,num_antena,num_subchannel]  
      data = np.transpose(data,axes=(1,0,2))
      return data
   
   @staticmethod
   def mods_to_one_hot (mods,inp,encoded):
     
    return encoded[mods.index(inp)]
   
   @staticmethod
   def MlP(input_shape,output_shape,
                      num_layers = [2**10,2**12,2**12,64*32*2],
                      k_r = None):
     
      num_last_layer = output_shape[0]*output_shape[1]*output_shape[2]
      input_signal = Input(shape = input_shape) 
      flatten =Flatten()(input_signal)
      for i,num in enumerate(num_layers[1:-1]):
        if i==0: dense = Dense(num, activation="relu" , kernel_regularizer = k_r)(flatten)
        dense = Dense(num, activation="relu" , kernel_regularizer = k_r)(dense)

      last_dense = Dense(num_last_layer, activation=None , kernel_regularizer = k_r)(dense)    
      out_put = Reshape(output_shape, input_shape=(last_dense))(last_dense)  
      return Model(inputs=input_signal,outputs=out_put) 
 

   @staticmethod
   def MlP_with_dropouts(input_shape,output_shape,
                      num_layers = [2**10,2**12,2**12,64*32*2],
                      dropout_rate=.1 ,
                      k_r = None):
     
      num_last_layer = output_shape[0]*output_shape[1]*output_shape[2]
      input_signal = Input(shape = input_shape) 
      flatten =Flatten()(input_signal)
      for i,num in enumerate(num_layers[1:-1]):
        if i==0: dense = Dense(num, activation="relu" , kernel_regularizer = k_r)(flatten)
        dense = Dense(num, activation="relu" , kernel_regularizer = k_r)(dense)
        drop = Dropout(dropout_rate)(dense)

      last_dense = Dense(num_last_layer, activation=None , kernel_regularizer = k_r)(drop)    
      out_put = Reshape(output_shape, input_shape=(num_last_layer))(last_dense)   
      return Model(inputs=input_signal,outputs=out_put) 


   @staticmethod
   def complex_to_array(dataset):
      shape = dataset.shape
      temp = dataset.reshape([*shape,1])
      real_num , imag_num = np.real(temp) , np.imag(temp)
      array_form_of_complex = np.concatenate((real_num,imag_num),axis=len(shape))
      return array_form_of_complex 

   @staticmethod
   def transform(x,mean,scale):
     return (x-mean)/scale


   @staticmethod 
   def mean_data(data):
    for i in range(len(data.shape)-2):
        if i == 0 : mean = np.mean(data,axis=0)
        mean = np.mean(mean,axis=0)   
    return mean
    
   @staticmethod 
   def scale(data):
    max = np.max(np.abs(data))
    return  max 


   @staticmethod  
   def make_mask (dataset , inp_mask = None , num_set_antenna_M = 8):
    num_ant ,num_subchannel = dataset.shape[0:2];
    mask = np.zeros(dataset.shape);

    if inp_mask == None : 
        antena_selected_mask = np.random.choice(num_ant,
                                                num_set_antenna_M,
                                                replace=False)
        
    else: antena_selected_mask = np.where( mask == 1)
    mask[ antena_selected_mask] =  1
    return mask

   @staticmethod
   def plot_imshow(*atr):
    fig = make_subplots(rows=len(atr), cols=2)
    for j in range(len(atr)):
        for i in range(2):
            fig.add_trace(px.imshow(atr[j][:,:,i]).data[0], row=j+1,col=i+1)
    fig.show()


   @staticmethod
   def NMSE_loss (y_true , y_pred):
    return tf.cast(K.mean(K.square(y_pred - y_true)/ (2*K.square(y_true))),tf.float64)

   @staticmethod
   def MSE_loss (y_true , y_pred):
    return K.mean(K.square(y_pred-y_true))

  
   def get_MIMO_data(self,scenario ) :

     # Load the default parameters
      parameters = DeepMIMO.default_params()
      # Set scenario name
      parameters['scenario'] = scenario
      # Set the main folder containing extracted scenarios
      parameters['dataset_folder'] = r''+ self.data_path
      # To activate the basestations from 1 tiil 64, set
      # each antena has 8 RX 
      parameters['active_BS'] = np.arange(1,self.active_BS + 1)
      # To activate the user rows 1-5 
      # according to practical examination i2_2p4 has 201 in each column 
      parameters['user_row_first'] = self.user_row_first
      parameters['user_row_last'] = self.user_row_last


      # Number of BS antennas in (x, y, z)
      parameters['bs_antenna']['shape'] = np.array([1, 1, 1])
      # To generate channels at 0.02 GHz  bandwidth, set
      parameters['OFDM']['bandwidth'] = self.bandwidth
      # To generate OFDM channels with 64 subcarriers, set
      parameters['OFDM']['subcarriers'] = self.subcarriers
      # To sample first 16 subcarriers by every spacing between each, set
      parameters['OFDM']['subcarriers_sampling'] = self.subcarriers_sampling
      #according to the picture in the article we chioced 32 instead of 16, that was mentioned in TABLE 1
      parameters['OFDM']['subcarriers_limit'] = self.subcarriers_limit
      # To only include 1 strongest paths in the channel computation, set
      parameters['num_paths'] = self.num_paths

      #Note: Since this scenario consists 
      #of only one access point, please set “enable_BS2BSchannels”
      #(MATLAB) or “enable_BS2BS” (Python) to zero in the DeepMIMO 
      #generation parameters. 
      parameters['enable_BS2BS'] = 0

      # Generate data
      dataset = DeepMIMO.generate_data(parameters)

      #delete unnccessary info
        #find keys in dictionary which is unneccessary
      dataset_keys = list(dataset[0].keys())
      dataset_user_keys = list(dataset[0]['user'].keys())
        #delte those keys 
      for i in range(len(dataset)):
          for del_key in [key for key in dataset_keys if key is not 'user']:
              del dataset[i][del_key]
          for del_key in [key for key in dataset_user_keys if key is not 'channel']:
              del dataset[i]['user'][del_key]

      return dataset

mim.y_train.shape

## PE

In [8]:
%cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [9]:
from generate_model import Designed_model as GM
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Reshape,Dense, GaussianNoise,Lambda,Dropout
from keras.models import Model
from keras import regularizers
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras import layers , Input
from tensorflow.keras.layers import Dense , LSTM ,Dropout , Activation ,Reshape

class PE(GM):
   def __init__(self,data_path):
      super(PE, self).__init__(data_path)



   def build_model_and_get_data(self):

      if type(self.X_train) == type(None) : self.set_data() 

      self.model = self.RNN_model ( self.X_train[0].shape, 
                                    self.y_train[0].shape[0])
      
      self.compile_model(self.model)
      
      return 
   


   def train_gpu(self,epochs =20,batch_size=1024 ,compile = True): 
       with tf.device(tf.test.gpu_device_name()):
          self.train(epochs=epochs, batch_size=batch_size)    

   def train(self, epochs =20,batch_size=1024 ,compile = True, normalize = True , get_data_build =True):

      if get_data_build : self.build_model_and_get_data()
      if compile : self.model = self.compile_model(self.model)
      if normalize : self.norm ()

      history = self.model.fit(self.X_train,self.y_train,
              validation_data=(self.X_test, self.y_test),
              epochs=epochs, batch_size = batch_size, verbose=2 )
      
      return history
   
   def norm(self):

      self._mean = self.mean_data(np.array([self.mean_data(self.X_train),
                                          self.mean_data(self.X_test)]))

      self._scale = max(self.scale(self.X_train),
                        self.scale(self.X_test))
      
      for value in [self.X_train,self.X_test]:
        value = self.transform(value,self._mean,self._scale)

   
   
   
   def set_data(self , limit = 500000 , execute = False)  -> None:

      train_batch_size = 128
      train_snr = np.arange(20, -4, -2)
      test_snr = np.arange(0, 6.5, 0.5)
      train_ratio = np.array([0.4, 0.6, 0.8, 1.0])
      epoch_setting = np.array([10**1, 10**2, 10**3, 10**4, 10**5])

      if  execute : 
        os.mkdir(self.data_path)
        exec(open(self.data_path + 'deep-neural-network-decoder/RNN/noise/K_16_N_32/"train_data_10^6"/get_data.py'))

        ##### execute train ##########
        X_train, y_train = self.get_data_by_ratio (train_ratio = 0.8 ,
                                                   snr = train_snr,status = "train",
                                                   limit= limit,
                                                   data_path = self.data_path + "data/")    
        
        np.savez_compressed( self.data_path + 'train_shape', X_train_shape= np.array(X_train.shape) , y_train_shape = np.array(y_train.shape))
        print("train data shape saved")
        np.savez_compressed( self.data_path + 'train', X_train= X_train.reshape(-1,1), y_train = y_train.reshape(-1,1))
        print("train data saved")
        del X_train, y_train

        ##### execute test ##########
        X_test, y_test = self.get_data_by_ratio (train_ratio = 0.8 ,
                                                   snr = train_snr,status = "train",
                                                   limit= int(limit*0.2),
                                                   data_path = self.data_path + "data/",
                                                 ) 

        np.savez_compressed( self.data_path + 'test_shape', X_test_shape= np.array(X_test.shape) , y_test_shape = np.array(y_test.shape))
        print("test data shape saved")
        np.savez_compressed( self.data_path + 'test', X_test= X_test.reshape(-1,1), y_test = y_test.reshape(-1,1))
        print("test data saved")
        del X_test, y_test

      ###### Load data from npy saved based on exceuting data earlier ########
      print("Loading Train Data...")
      loaded = np.load( self.data_path + 'train.npz')
      loaded_shape = np.load( self.data_path + 'train_shape.npz')
      X_train_shape , y_train_shape = loaded_shape["X_train_shape"] , loaded_shape["y_train_shape"] 
      self.X_train , self.y_train = loaded["X_train"].reshape(X_train_shape) , loaded["y_train"].reshape(y_train_shape)


      print("Loading test Data...")
      loaded = np.load( self.data_path + 'test.npz')
      loaded_shape = np.load( self.data_path + 'test_shape.npz')
      X_test_shape , y_test_shape = loaded_shape["X_test_shape"] , loaded_shape["y_test_shape"] 
      self.X_test , self.y_test = loaded["X_test"].reshape(X_test_shape) , loaded["y_test"].reshape(y_test_shape)
      print("loading is finished ")
      del loaded
      # make the data independent of snr 
      self.X_train , self.y_train = self.X_train.reshape(-1,X_train_shape[-1]) , self.y_train.reshape(-1,y_train_shape[-1])
      self.X_test , self.y_test = self.X_test.reshape(-1,X_test_shape[-1]) , self.y_test.reshape(-1,y_test_shape[-1])
      
   
   
   @staticmethod
   def transform(x,mean,scale):
     return (x-mean)/scale

   @staticmethod
   def mean_data(data):
      mean = np.mean(data,axis=0)
      for i in range(1,len(data.shape)-2):
        mean = np.mean(mean,axis=0)   
      return mean

   @staticmethod 
   def scale(data):
      max = np.max(np.abs(data))
      return max 



   @staticmethod
   def compile_model(model):
      model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=0.001),
                    loss= tf.keras.losses.MeanSquaredError(),
                    metrics=["accuracy"],
                )
      return model
   

   @staticmethod
   def RNN_model (input_shape,output_shape, Dropout_rate = 0.1) :
  
    input_signal = Input(shape = input_shape)
    reshape = Reshape((*input_shape,1))(input_signal)
    lstm = LSTM(256,
                dropout = Dropout_rate,
                kernel_initializer= tf.keras.initializers.GlorotNormal(seed=None))(reshape)

    output = Dense(output_shape,"sigmoid")(lstm)
    
    return Model(inputs = input_signal, outputs = output)


    @staticmethod
    def get_data_by_ratio (train_ratio, snr, status = "train", limit= None,
                      data_path = "/content/project_3/PE/data/"):
      
      if status == "train" : data = sio.loadmat(data_path+"ratio_0.4_train_snr_8dB")
      elif status == "test" : data = sio.loadmat(data_path+'test_snr_0.0dB')
      x_shape = data['x_'+status].shape
      y_shape = data['y_'+status].shape
      x_shape = [min(x_shape[0],limit) , x_shape[1]]
      y_shape = [min(y_shape[0],limit) , y_shape[1]]

      x = np.zeros([len(snr),*x_shape])
      y = np.zeros([len(snr),*y_shape])
      print(x_shape)
      del data
    
      for i, tr_snr in enumerate(snr):
          print("added {} SNR".format(tr_snr))
          if status == "train" : filename = 'ratio_' + str(train_ratio) + '_train_snr_' + str(tr_snr) + 'dB'
          elif status == "test" : filename = 'test_snr_' + str(tr_snr) + 'dB'
          data = sio.loadmat(data_path+filename)
          x[i,:,:] = data['x_'+status][:limit,:]
          y[i,:,:] = data['y_'+status][:limit,:]

      return [x , y]






## MIST

In [10]:
import tensorflow as tf
import numpy as np
from keras.layers.core import Dense
from keras.layers import Conv1D,Flatten, BatchNormalization, Input
from keras import backend as K
from keras.models import Model
from keras.callbacks import ModelCheckpoint


from generate_model import Designed_model as GM
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Reshape,Dense, GaussianNoise,Lambda,Dropout
from keras.models import Model
from keras import regularizers
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

class MIST():
   def __init__(self, k = 500 ,
                      rate = 1/2 ,
                      SNR_dB_start_Eb = -1 ,
                      SNR_dB_stop_Eb = 7 ,
                      SNR_points = 9 ,
                      n_samples = 10 , 
                      train_batch = 1024,
                      test_batch = 1024,
                      g1=[1,1,1],
                      g2=[1,0,1]):
     
      #dataword length  => k  
      #encoder rate is used => rate 
      self.N = int(k/rate)  #codeword length
      for i,(key,value) in enumerate(locals().items()):
          if i < 2 : continue
          setattr(self,key,value)
   
   def build_model(self):
      self.model = self.MIST_model(input_shape=(self.N,1),output_shape=(self.k)) 
      return self.model

   def build_model_and_get_data(self):
      self.build_model()
      self.X_train , self.y_train = self.get_data()
      self.compile_model() 
   
   def train_gpu(self,epochs = 200 , batch_size=1024 ,compile = True): 
       with tf.device(tf.test.gpu_device_name()):
          self.train(epochs=epochs, batch_size=batch_size)    
 
   def train(self, epochs = 200, batch_size=1024 ,MIST_train = True ,compile = True, get_data_build =True):
      self.n_samples = int(epochs/self.SNR_points) 
      self.build_model()
      self.compile_model()
      
      if MIST_train : 
        history = self.MIST_train_idea()

      
      else :
          X_train , y_train = self.get_data()
          history = self.model.fit(X_train,y_train,
                epochs = 1, batch_size = batch_size, verbose=2 )
      return history
   
   
   def get_data(self , limit = None ) :
      sigmas = self. get_sigmas(self.SNR_dB_start_Eb,
                                self.SNR_dB_stop_Eb,
                                self.SNR_points,
                                self.k,
                                self.N)
      
      X,y = self.generate_uncode_noisy_signal(sigmas)
      return  [X,y]

  
   

   
   def generate_uncode_noisy_signal(self,sigmas):
     
     X = np.zeros([len(sigmas),self.n_samples,self.train_batch,self.N,1])
     y = np.zeros([len(sigmas),self.n_samples,self.train_batch,self.k])
     for i in range(0,self.n_samples): 
        for ii in range(0,len(sigmas)):
            #Generating dataword and codeword
            uncoded = np.random.randint(0,2,size=(self.train_batch,self.k))
            encoded = np.zeros([uncoded.shape[0], self.N])
            # memory 2 convolutional encoder used g1, g2
            for iii in range(0,self.train_batch): 
                encoded[iii,:] = self.convenc (uncoded[iii,:],self.g1,self.g2,self.k)
            #Modulate
            signal = 2*encoded - 1
            #Adding noise
            noisy_signal = signal + np.random.normal(0, sigmas[i], size= np.shape(signal))
            ## train 
            X[i,ii,:,:,0] = noisy_signal
            y[i,ii,:,:] = uncoded
      
     return [ X.reshape(-1,self.N) , y.reshape(-1,self.k)]




   def MIST_train_idea(self , limit = None ):
     sigmas = self.get_sigmas(self.SNR_dB_start_Eb,
                                self.SNR_dB_stop_Eb,
                                self.SNR_points,
                                self.k,
                                self.N)
     history = [] 
     for i in range(0,len(sigmas)):
       print("SNR  {}/{}".format(i,len(sigmas)))
       for ii in range(0,self.n_samples):
            #Generating dataword and codeword
            uncoded = np.random.randint(0,2,size=(self.train_batch,self.k))
            encoded = np.zeros([uncoded.shape[0], self.N])
            # memory 2 convolutional encoder used g1, g2
            for iii in range(0,self.train_batch): 
                encoded[iii,:] = self.convenc(uncoded[iii,:],self.g1,self.g2,self.k)
            #Modulate
            signal = 2*encoded - 1
            #Adding noise
            noisy_signal = signal + np.random.normal(0, sigmas[i], size= np.shape(signal))
            ## train l
            print("epoch  {}/{}".format(ii,self.n_samples))
            x_train = np.expand_dims(noisy_signal,axis=2)
            history.append( self.model.fit(x_train,uncoded, epochs = 1, batch_size = self.train_batch , verbose=2))

     return history 

   def compile_model(self, optimizer = 'adam',loss = 'mse' ):            
      self.model.compile(optimizer=optimizer, loss=loss, metrics=[self.ber]) 
      return self.model

   
   @staticmethod
   def get_sigmas(SNR_dB_start_Eb,SNR_dB_stop_Eb,SNR_points,k,N):
      
      SNR_dB_start_Es = SNR_dB_start_Eb + (10*np.log10(1.0*k/N))
      SNR_dB_stop_Es = SNR_dB_stop_Eb + (10*np.log10(1.0*k/N))
      SNR_range=np.linspace(SNR_dB_start_Es, SNR_dB_stop_Es, SNR_points)
      return np.sqrt(1/(2*10**(SNR_range/10)))
   
   
   @staticmethod
   def convenc(data,g1,g2,k):
    enc_msg = np.zeros([2*k])
    enc_msg[0::2] = (np.convolve(data,g1)%2)[0:k]
    enc_msg[1::2] = (np.convolve(data,g2)%2)[0:k]
    return enc_msg   

    
   @staticmethod
   def ber(y_true, y_pred):
     return  K.mean(K.cast(K.not_equal(y_true, K.round(y_pred)),dtype='float32'))
   
   @staticmethod
   def MIST_model(input_shape , output_shape):  
      input_batch=Input(shape=input_shape)
      conv1 = Conv1D(10, 24, activation='relu',padding='same')(input_batch)
      batch_norm1=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv1)
      conv2 = Conv1D(50, 24, activation='relu',padding='same')(batch_norm1)
      batch_norm2=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv2)
      conv3 = Conv1D(50, 24, activation='relu',padding='same')(batch_norm2)
      batch_norm3=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv3)
      flatten = Flatten()(batch_norm3)
      msg_out = Dense(output_shape, activation='sigmoid')(flatten)

      return Model(inputs=input_batch,outputs=msg_out)
  

# test

In [11]:
a = aphl("/content/gdrive/MyDrive/RML2016.10a/")

In [12]:
p = PE("/content/gdrive/MyDrive/project_3/PE")

In [13]:
mim = Deep_MIMO('/content/gdrive/MyDrive/DeepMIMO2',active_BS=20,user_row_last=2)

In [41]:
mis = MIST()

In [14]:
#a.train_gpu()

Epoch 1/20
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/RML2016.10a/assets
129/129 - 17s - loss: 2.0855 - accuracy: 0.2224 - val_loss: 1.9215 - val_accuracy: 0.2810 - 17s/epoch - 134ms/step
Epoch 2/20
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/RML2016.10a/assets
129/129 - 5s - loss: 1.8183 - accuracy: 0.3151 - val_loss: 1.7424 - val_accuracy: 0.3368 - 5s/epoch - 40ms/step
Epoch 3/20
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/RML2016.10a/assets
129/129 - 4s - loss: 1.6929 - accuracy: 0.3580 - val_loss: 1.6548 - val_accuracy: 0.3715 - 4s/epoch - 35ms/step
Epoch 4/20
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/RML2016.10a/assets
129/129 - 5s - loss: 1.6240 - accuracy: 0.3801 - val_loss: 1.6185 - val_accuracy: 0.3834 - 5s/epoch - 41ms/step
Epoch 5/20
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/RML2016.10a/assets
129/129 - 5s - loss: 1.5608 - accuracy: 0.3959 - val_loss: 1.5374 - val_accuracy: 0.3968 - 5s/epoch - 35

In [ ]:
#mim.train()

In [ ]:
#p.train()

In [14]:
a.build_model_and_get_data()

In [20]:
def show (m):
  print('train shape : {}'.format(m.X_train.shape))
  print('train shape : {}'.format(m._mean))
  print("model {}".format(m.model.summary()))  

In [ ]:
!pip3 install foolbox==3.1.1
# !pip3 install git+https://github.com/bethgelab/foolbox.git

In [ ]:
!pip3 install --upgrade tensorflow

In [21]:
show(a)

train shape : (132000, 2, 128)
train shape : 0.1642291247844696
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2, 128)]          0         
                                                                 
 reshape (Reshape)           (None, 2, 128, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 1, 121, 128)       2176      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 60, 128)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 45, 64)         131136    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 22, 64)        0       

In [22]:
import foolbox as fb
model = tf.keras.applications.ResNet50(weights="imagenet")
preprocessing = dict(flip_axis=-1, mean=[103.939, 116.779, 123.68])
bounds = (0, 255)
fmodel = fb.TensorFlowModel(model, bounds=bounds, preprocessing=preprocessing)
images, labels = fb.utils.samples(fmodel, dataset='imagenet', batchsize=16)
fb.utils.accuracy(fmodel, images, labels)

0.875

In [24]:
images.shape

TensorShape([16, 224, 224, 3])

In [26]:
import foolbox as fb
model = a.model
preprocessing = dict()
bounds = (np.min(a.X_train), np.max(a.X_train))
fmodel = fb.TensorFlowModel(a.model, bounds=bounds,preprocessing=preprocessing)
fb.utils.accuracy(fmodel, inpu[:17], a.y_train[:17])
#attack = fb.attacks.LinfDeepFoolAttack()
#raw, clipped, is_adv = attack(fmodel, mim.X_train, mim.y_train, epsilons=0.03)

NameError: ignored

In [26]:
i = tf.convert_to_tensor(a.X_train, np.float64)

In [98]:
model = tf.keras.applications.ResNet50(weights="imagenet")
model